In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
#Import dataframes 
df_player_stats_100pos = pd.read_csv('DataFiles/player_stats_100pos.csv')
df_bio = pd.read_csv('DataFiles/df_bio.csv')
data_index = pd.read_csv('DataFiles/data_index.csv')

In [3]:
columns_to_delete = ['PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK',
                     'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK',
                     'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK',
                     'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK',
                     'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK',
                     'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK',
                     'DD2_RANK', 'TD3_RANK', 'WNBA_FANTASY_PTS_RANK']

# Delete the specified columns
df_player_stats_100pos.drop(columns=columns_to_delete, inplace=True, errors='ignore')

df_player_stats_100pos.rename(columns={'Season_type': 'Season'}, inplace=True)

In [4]:
# Columns to be dropped in 'df_bio' table:
df_bio.drop(columns=['PLAYER_HEIGHT', 'PLAYER_HEIGHT_INCHES', 'PLAYER_WEIGHT', 'PTS', 'REB', 'AST'], inplace=True, errors='ignore')

In [5]:
# Merge 'df_player_stats' and ‘df_bio‘ on common keys ‘PLAYER_ID‘ and ‘Year‘ in order to get 'COUNTRY' column.
merged_df_stats_bio = pd.merge(df_player_stats_100pos, df_bio, on=['PLAYER_ID', 'Year', 'Season'], how='inner')

columns_to_delete_merged = ['GP_y','NET_RATING','OREB_PCT','DREB_PCT','USG_PCT', 'TS_PCT','AST_PCT']

merged_df_stats_bio.drop(columns=columns_to_delete_merged, inplace=True)      

In [7]:
# Checking the result of 'merged_df_stats_bio'
merged_df_stats_bio.shape

(18282, 42)

In [8]:
# Merging 'merged_df_stat_bio' with player index table on player-id to get player´s role/position.
merged_df_100pos = pd.merge(merged_df_stats_bio, data_index, left_on='PLAYER_ID', right_on='PERSON_ID', how='inner')

# Dropping all duplicate columns with trailing letter 'y'.
columns_to_drop = [col for col in merged_df_100pos.columns if 'y' in col]
merged_df_100pos.drop(columns=columns_to_drop, inplace=True)

# Dropping redundant columns
columns_to_drop_2 = [
    'PLAYER_LAST_NAME',
    'PLAYER_FIRST_NAME',
    'PLAYER_SLUG',
    'TEAM_ID',
    'TEAM_SLUG',
    'IS_DEFUNCT',
    'TEAM_NAME',
    'NICKNAME',
    'TEAM_ABBREVIATION',
    'ROSTER_STATUS',
    'STATS_TIMEFRAME', 
    'PERSON_ID',
    'TEAM_CITY'
]

merged_df_100pos.drop(columns=columns_to_drop_2, inplace=True)

In [11]:
# Drop '_x' at the end of column names 
for column in merged_df_100pos.columns: 
    if '_x' in column: 
        column_name = column.replace('_x', '')
        merged_df_100pos.rename(columns={column:column_name}, inplace=True)

# Change regular season name at 'Season' column  
merged_df_100pos['Season'] = merged_df_100pos['Season'].apply(lambda x:'Regular Season' if x =='Regular%20Season' else x)

In [12]:
# Output of final dataframe
nj = merged_df_100pos[(merged_df_100pos['PLAYER_NAME'] == 'Arvydas Sabonis')]
nj

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,Year,Season,COLLEGE,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,JERSEY_NUMBER,POSITION,HEIGHT,WEIGHT,FROM_YEAR,TO_YEAR
286,717,Arvydas Sabonis,1610612757,POR,32.0,69,43,26,0.623,51.3,9.6,19.2,0.498,1.4,3.8,0.371,6.5,8.4,0.777,3.3,12.6,15.9,4.3,4.4,1.8,2.4,1.0,5.9,0.0,27.0,1996-97,Regular Season,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
287,717,Arvydas Sabonis,1610612757,POR,32.0,4,1,3,0.250,53.9,9.0,21.1,0.429,1.0,4.0,0.250,3.5,4.0,0.875,4.0,9.0,13.1,4.5,4.5,1.5,1.5,2.5,9.5,4.0,22.6,1996-97,Playoffs,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
288,717,Arvydas Sabonis,1610612757,POR,33.0,73,40,33,0.548,52.1,9.1,18.5,0.493,0.7,2.6,0.261,7.2,9.1,0.798,3.3,13.0,16.3,4.9,4.2,1.5,1.8,1.0,6.0,0.0,26.1,1997-98,Regular Season,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
289,717,Arvydas Sabonis,1610612757,POR,33.0,4,1,3,0.250,53.0,9.0,19.9,0.450,0.5,1.0,0.500,6.0,7.0,0.857,3.5,11.9,15.4,3.0,5.0,3.5,1.5,3.5,9.5,0.0,24.4,1997-98,Playoffs,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
290,717,Arvydas Sabonis,1610612757,POR,34.0,50,35,15,0.700,53.3,9.2,18.9,0.485,0.3,0.9,0.292,5.3,6.9,0.771,3.5,12.1,15.5,4.7,3.4,1.3,2.5,1.3,5.8,0.0,23.9,1998-99,Regular Season,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
291,717,Arvydas Sabonis,1610612757,POR,34.0,13,7,6,0.538,55.9,6.4,16.1,0.398,0.1,0.7,0.200,5.6,6.1,0.907,1.7,14.6,16.3,4.1,2.7,2.1,2.1,1.4,5.0,0.0,18.6,1998-99,Playoffs,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
292,717,Arvydas Sabonis,1610612757,POR,35.0,66,50,16,0.758,52.2,9.4,18.5,0.505,0.2,0.6,0.368,5.2,6.1,0.843,3.0,12.9,15.9,3.7,3.0,1.3,2.4,0.7,5.7,0.4,24.1,1999-00,Regular Season,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
293,717,Arvydas Sabonis,1610612757,POR,35.0,16,10,6,0.625,55.8,7.7,17.0,0.453,0.7,2.4,0.286,4.4,5.6,0.796,2.2,10.0,12.1,3.5,2.9,1.6,1.5,1.0,6.7,0.3,20.5,1999-00,Playoffs,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
294,717,Arvydas Sabonis,1610612757,POR,36.0,61,37,24,0.607,52.1,9.9,20.8,0.479,0.0,0.6,0.067,4.9,6.3,0.776,2.1,11.3,13.3,3.7,3.4,1.6,2.5,1.0,5.5,0.0,24.8,2000-01,Regular Season,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002
295,717,Arvydas Sabonis,1610612757,POR,36.0,3,0,3,0.000,51.8,6.9,14.4,0.483,0.0,1.0,0.000,3.0,4.0,0.750,4.0,8.4,12.4,4.0,1.0,0.5,3.5,0.5,5.9,0.0,16.8,2000-01,Playoffs,NaN,Lithuania,1986,1,24,11,C,7-3,292.0,1995,2002


In [13]:
merged_df_100pos.shape

(18282, 43)

In [17]:
#Export dataframe 

merged_df_100pos.to_csv('merged_df_100pos.csv', index=False)